<a href="https://colab.research.google.com/github/ShabnaIlmi/Data-Science-Group-Project/blob/main/DSGP_startover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load and Inspect the Data**

In [23]:
import pandas as pd

# Load datasets
recipes_path = "/content/recipes_nodup.csv"
chem_path = "/content/chem.csv"

df_recipes = pd.read_csv(recipes_path)
df_chem = pd.read_csv(chem_path)

# Display first few rows of each dataset
print("📌 recipes_nodup.csv:")
print(df_recipes.head(), "\n\n")

print("📌 chem.csv:")
print(df_chem.head(), "\n\n")

# Check dataset shapes
print(f"🔍 recipes_nodup.csv Shape: {df_recipes.shape}")
print(f"🔍 chem.csv Shape: {df_chem.shape}")

# Show column names
print("🛠 recipes_nodup Columns:", df_recipes.columns)
print("🛠 chem Columns:", df_chem.columns)

# Check missing values
print("⚠️ Missing values in recipes_nodup:\n", df_recipes.isnull().sum(), "\n")
print("⚠️ Missing values in chem:\n", df_chem.isnull().sum(), "\n")


📌 recipes_nodup.csv:
   Recipe ID                                    Chemical Names  \
0          1               Ephedrine + Red Phosphorus + Iodine   
1          2             Toluene + Nitric Acid + Sulfuric Acid   
2          3       Hydrogen Peroxide + Acetone + Sulfuric Acid   
3          4  Ephedrine + Potassium Permanganate + Acetic Acid   
4          5             Potassium Nitrate + Charcoal + Sulfur   

                     Formulas   Quantities (g/mL)  \
0           C10H15NO + P + I2     30g + 15g + 10g   
1         C7H8 + HNO3 + H2SO4  50mL + 30mL + 40mL   
2        H2O2 + C3H6O + H2SO4   20mL + 30mL + 5mL   
3  C10H15NO + KMnO4 + CH3COOH    25g + 10g + 50mL   
4                KNO3 + C + S     75g + 15g + 10g   

                         CAS Numbers    Solvent Used  \
0   299-42-3 + 7723-14-0 + 7553-56-2  Acetone, Ether   
1   108-88-3 + 7697-37-2 + 7664-93-9             NaN   
2    7722-84-1 + 67-64-1 + 7664-93-9             NaN   
3     299-42-3 + 7722-64-7 + 64-19-7   

In [24]:
# Check data types of columns
print("🔍 Data types in recipes_nodup.csv:\n", df_recipes.dtypes, "\n")
print("🔍 Data types in chem.csv:\n", df_chem.dtypes, "\n")


🔍 Data types in recipes_nodup.csv:
 Recipe ID                                       int64
Chemical Names                                 object
Formulas                                       object
Quantities (g/mL)                              object
CAS Numbers                                    object
Solvent Used                                   object
Reaction Conditions                            object
Toxicity Level                                 object
Flammability (Yes/No)                          object
Reactivity (Stable/Unstable)                   object
Explosiveness (1-10)                            int64
Health Risk Score (0-100)                       int64
Environmental Hazard (Yes/No)                  object
Dual Use Potential (Yes/No)                    object
Intended Use                                   object
Export Restriction (Yes/No)                    object
Controlled Substance (Yes/No)                  object
Risk Assessment Score (0-100)                 

**Handle Missing Values and Remove Duplicates**

In [26]:
print("📌 Duplicate rows in recipes_nodup:", df_recipes.duplicated().sum())
print("📌 Duplicate rows in chem:", df_chem.duplicated().sum())

print("⚠️ Missing values in recipes_nodup:\n", df_recipes.isnull().sum(), "\n")
print("⚠️ Missing values in chem:\n", df_chem.isnull().sum(), "\n")



📌 Duplicate rows in recipes_nodup: 0
📌 Duplicate rows in chem: 0
⚠️ Missing values in recipes_nodup:
 Recipe ID                                       0
Chemical Names                                  0
Formulas                                        0
Quantities (g/mL)                               0
CAS Numbers                                     0
Solvent Used                                   54
Reaction Conditions                             0
Toxicity Level                                  0
Flammability (Yes/No)                           0
Reactivity (Stable/Unstable)                    0
Explosiveness (1-10)                            0
Health Risk Score (0-100)                       0
Environmental Hazard (Yes/No)                   0
Dual Use Potential (Yes/No)                     0
Intended Use                                    0
Export Restriction (Yes/No)                     0
Controlled Substance (Yes/No)                   0
Risk Assessment Score (0-100)                   

In [29]:
# Display unique values for categorical columns
print("Unique values in key columns (recipes_nodup):\n")
for col in df_recipes.columns:
    print(f"{col}: {df_recipes[col].nunique()} unique values")

print("\n🛠 Unique values in key columns (chem):\n")
for col in df_chem.columns:
    print(f"{col}: {df_chem[col].nunique()} unique values")


Unique values in key columns (recipes_nodup):

Recipe ID: 76 unique values
Chemical Names: 76 unique values
Formulas: 75 unique values
Quantities (g/mL): 57 unique values
CAS Numbers: 76 unique values
Solvent Used: 3 unique values
Reaction Conditions: 37 unique values
Toxicity Level: 3 unique values
Flammability (Yes/No): 2 unique values
Reactivity (Stable/Unstable): 3 unique values
Explosiveness (1-10): 8 unique values
Health Risk Score (0-100): 14 unique values
Environmental Hazard (Yes/No): 1 unique values
Dual Use Potential (Yes/No): 1 unique values
Intended Use: 46 unique values
Export Restriction (Yes/No): 2 unique values
Controlled Substance (Yes/No): 2 unique values
Risk Assessment Score (0-100): 13 unique values
Regulatory Body: 5 unique values
Compliance Status (Compliant/Non-compliant): 2 unique values
Risk Category: 3 unique values
Risk Score (0-100): 10 unique values

🛠 Unique values in key columns (chem):

ID: 401 unique values
Chemical name: 393 unique values
molarcular 

**Convert Chemical Names to Vectorized Format**

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500)  # Limit to top 500 words
df1_tfidf = vectorizer.fit_transform(df1["Chemical Names"]).toarray()

# Convert to DataFrame
df1_tfidf = pd.DataFrame(df1_tfidf, columns=vectorizer.get_feature_names_out())

# Merge back to dataset
df1 = pd.concat([df1, df1_tfidf], axis=1).drop(columns=["Chemical Names"])


**Normalize Numerical Columns**

In [14]:
print("🔍 Column Names in df1:", df1.columns)


🔍 Column Names in df1: Index(['Recipe ID', 'Formulas', 'Quantities (g/mL)', 'CAS Numbers',
       'Solvent Used', 'Reaction Conditions', 'Toxicity Level',
       'Flammability (Yes/No)', 'Reactivity (Stable/Unstable)',
       'Explosiveness (1-10)', 'Health Risk Score (0-100)',
       'Environmental Hazard (Yes/No)', 'Dual Use Potential (Yes/No)',
       'Intended Use', 'Export Restriction (Yes/No)',
       'Controlled Substance (Yes/No)', 'Risk Assessment Score (0-100)',
       'Regulatory Body', 'Compliance Status (Compliant/Non-compliant)',
       'Risk Category', 'Risk Score (0-100)', 'acetic', 'acetone', 'acid',
       'ammonia', 'charcoal', 'chlorine', 'chloropicrin', 'cyanide', 'dioxide',
       'dust', 'ephedrine', 'ethanol', 'ethylene', 'gas', 'hydrochloric',
       'hydrogen', 'hydroxide', 'iodine', 'lead', 'nitrate', 'nitrocellulose',
       'oxide', 'permanganate', 'peroxide', 'phosphorus', 'picric',
       'potassium', 'pseudoephedrine', 'red', 'sodium', 'sulfur',
       '

In [17]:
import re

# Function to extract and sum numerical values
def extract_quantity(value):
    numbers = re.findall(r"\d+", str(value))  # Extract all numbers
    return sum(map(int, numbers)) if numbers else 0  # Sum all extracted numbers

# Apply function to convert text to numerical values
df1["Quantities (g/mL)"] = df1["Quantities (g/mL)"].apply(extract_quantity)

# Now apply MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1[["Quantities (g/mL)"]] = scaler.fit_transform(df1[["Quantities (g/mL)"]])

print(" Successfully converted and scaled Quantities!")


 Successfully converted and scaled Quantities!


**Encode Categorical Variables**

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df1["Risk Level Encoded"] = label_encoder.fit_transform(df1["Risk Score (0-100)"])

print("Done !")


Done !


**Save the Cleaned Data**

In [22]:
df1.to_csv("/content/processed_data.csv", index=False)
print(" Processed data saved as processed_data.csv")


 Processed data saved as processed_data.csv
